# Data Science From Scratch Notes

## Chapter 7: Hypothesis and Inference

## Statistical Hypothesis Testing

Null hypothesis and alternative hypothesis.

### Example: Flipping a Coin

Let's test whether a coin is fair. We'll assume that the coin has some probability *p* of landing and heads, so our null hypothesis is that the coin is fair - $p = 0.5$. We'll test this agains the alternative hypothesis $p \neq 0.5$.

Each coin flip is a Bernoulli trial, which means that X is a Binomial(n,p) random variable, which we can approximate using the normal distribution:

In [1]:
from typing import Tuple
import math

def normal_approximation_to_binomial(n: int, p: float) -> Tuple[float, float]:
    """Returns mu and sigma corresponding to a Binomial(n,p)"""
    mu = n * p
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

We'll use `normal_cdf` to figure out the probability that a random variable's realized value lies within or outside a particular interval:

In [3]:
from scratch.probability import normal_cdf

# The normal cdf _is_ the probability the variable is below a threshold
normal_probability_below = normal_cdf

# It's above the threshold if it's not below the threshold
def normal_probability_above(lo: float,
                             mu: float = 0,
                             sigma: float = 1) -> float:
    """The probability that an N(mu, sigma) is greater than lo"""
    return 1 - normal_cdf(lo, mu, sigma)

# It's between if it's less than hi, but not less than lo
def normal_probability_between(lo: float,
                               hi: float,
                               mu: float = 0,
                               sigma: float = 1) -> float:
    """The probability that an N(mu, sigma) is between lo and hi"""
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# It's outside if it's not between
def normal_probability_outside(lo: float,
                               hi: float,
                               mu: float = 0,
                               sigma: float = 1) -> float:
    """The probability that an N(mu, sigma) is not betweeen lo and hi"""
    return 1 - normal_probability_between(lo, hi, mu, sigma)

We can also do the reverse - find either the nontail region or the symmetric interval around the mean that accounts for a certain level of likelihood:

In [4]:
from scratch.probability import inverse_normal_cdf

def normal_upper_bound(probability: float,
                       mu: float = 0,
                       sigma: float = 1) -> float:
    """Returns the z for which P(Z >= z) = probability"""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability: float,
                       mu: float = 0,
                       sigma: float = 1) -> float:
    """Returns the z for which P(Z >= z) = probability"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability: float,
                            mu: float = 0,
                            sigma: float = 1) -> float:
    """
    Returns the symmetric bounds that 
    contain the specified probability
    """
    tail_probability = (1 - probability) / 2
    
    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    
    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound

Let's say we choose to flip the coin $n= 1000$ times. If our hypothesis of fairness is tru, X should be distributed approximately normally with mean 500 and standard deviation 15.8:

In [7]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

print(mu_0)
print(sigma_0)

500.0
15.811388300841896


Next, we need to make a decision about significance - how willing we are to make a type I error ("false positive"), in which we reject $H_0$ even though it's true. Let's choose 5%. Consider the test that rejects $H_0$ if X falls outside the bounds given by:

In [8]:
# (469, 531)
lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)

print(lower_bound)
print(upper_bound)

469.01026640487555
530.9897335951244


Assuming *p* really equals 0.5 (i.e., $H_0$ is true), there is just a 5% chance we observe an X that lies outside this interval.

We can calculate the power of the test (1 - type 2 error ("false negative")):

In [9]:
# 95% bounds based on assumption p is 0.5
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)
print(lo)
print(hi)

# actual mu and sigma based on p = 0.55
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)
print(mu_1)
print(sigma_1)

# a type 2 error means we fail to reject the null hypothesis,
# which will happen when X is still in our originial interval
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability
print(power)

469.01026640487555
530.9897335951244
550.0
15.732132722552274
0.8865480012953671


What if our null hypothesis was that the coin is not biased toward heads, or that $p \leq 0.5$. In this case, we want a *one-sided test* that rejects the null hypothesis when X is much larger than 500 but not when X is smaller than 500.

A 5% significance test involves using `normal_probability_below` to find the cutoff below which 95% probability lies:

In [10]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
# is 526 (< 531, since we need more probability in the upper tail)

type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
print(power)

0.9363794803307173


## p-Values

Instead of choosing bounds based on some probability cutoff, we compute the probability - assuming $H_0$ is true - that we would see a value at least as extreme as the one we actually observed.

For a two-sided test of whether the coin is fair, we compute:

In [11]:
def two_sided_p_value(x: float, mu: float = 0, sigma: float = 1) -> float:
    """
    How likely are we to see a value at least as extreme as x (in either
    direction) if our vaues are from an N(mu, sigma)?
    """
    if x >= mu:
        # x is greater than the mean, so the tail is everything greater than x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # x is less than the mean, so the tail is everything less than x
        return 2 * normal_probability_below(x, mu, sigma)

If we were to see 530 heads, we would compute:

In [12]:
two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598835

We used a value of 529.5 rather than 530 as a continuity correction.

We can convince ourselves that this is a sensivle estimate with a simulation:

In [17]:
import random

random.seed(0)

extreme_value_count = 0
for _ in range(1000):
    num_heads = sum(1 if random.random() < 0.5 else 0 # Count # of heads
                    for _ in range(1000)) # in 1000 flips,
    if num_heads >= 530 or num_heads <= 470: # and count how often
        extreme_value_count += 1 # the # is 'extreme'
        
# p-value was 0.062 => ~62 extreme values out of 1000
assert 59 < extreme_value_count < 65, f"{extreme_value_count}"

Since the p-value is greater than our 5% significance, we don't reject the null. If we saw 532 heads, the p-value would be:

In [18]:
two_sided_p_value(531.5, mu_0, sigma_0)

0.046345287837786575

which is smaller than the 5% significance, which means we would reject the null. It's the same test as before, just a different way of approaching statistics.

Similarly, we have:

In [19]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

For our one-sided test, if we saw 525 heads we would compute:

In [20]:
upper_p_value(524.5, mu_0, sigma_0)

0.06062885772582072

which means we wouldn't reject the null. If we saw 527 heads, the computation would be:

In [21]:
upper_p_value(526.5, mu_0, sigma_0)

0.04686839508859242

and we would reject the null.

## Confidence Intervals

A third approach to testing hypotheses is confidence intervals.

Since we don't know the exact value of *p*, we use our estimate if we observe 525 heads out of 1,000 flips:

In [23]:
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)

normal_two_sided_bounds(0.95, mu, sigma)

(0.4940490278129096, 0.5559509721870904)

Interpretation: 95% of the time, the "true" value of *p* will lie between 0.49 and 0.56.

## p-Hacking

A procedure that erroneously rejects the null hypothesis only 5% of the time will, by definition, 5% of the time erroneously reject the null hypothesis:

In [24]:
from typing import List

def run_experiment() -> List[bool]:
    """Flips a fair coin 1000 times, True = heads, False = tails"""
    return[random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment: List[bool]):
    """using the 5% signicance level"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment
                      for experiment in experiments
                      if reject_fairness(experiment)])

assert num_rejections == 46

The implications: Test enough hypotheses against your dataset, and one of them will almost certainly appear significant. Remove the right outliers, and you can probably get your p-value below 0.05.

To do good *science*, determine the hypothesis before looking at the data, clean the data without the hypotheses in mind.


## Bayesian Inference

In the procedures we've looked at, we have made probability statements about our *tests*.

An alternative approach involves treating the unknown parameters themselves as random variables. We start with a *prior distribution* for the parameters and then use the observed data and Bayes' theorem to get an updated *posterior distribution* for the parameters. In this approach, we make probability judgements about the parameters rather than the tests.

When the unknown parameter is a probability (e.g. coin-flipping), we often use a prior from teh *Beta distribution*, which puts all its probability between 0 and 1:

In [25]:
def B(alpha: float, beta: float) -> float:
    """A normalizing constant so that the total probability is 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x: float, alpha: float, beta: float) -> float:
    if x <= 0 or x >= 1: # no weight outside of [0, 1]
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)

Generally, this distribution centers its weight at $$\frac{\alpha}{\alpha + \beta}$$ and the larger alpha and beta are, the "tighter" the distribution is.